In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

api_keys={
    'google_api_key':os.getenv('google_api_key'),
    'pse':os.getenv('pse'),
    'openai_api_key':os.getenv('openai_api_key'),
    'composio_key':os.getenv('composio_api_key')
}

In [2]:
from composio_langgraph import Action, ComposioToolSet, App
from utils.composio_tools_agent import Composio_agent
from pydantic_ai import Agent, format_as_xml,RunContext
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.providers.google import GoogleProvider
from langchain_openai import ChatOpenAI
from datetime import datetime
from pydantic_ai.messages import ModelMessage
from pydantic import BaseModel,Field
from dataclasses import dataclass
import nest_asyncio
nest_asyncio.apply()

In [25]:


@dataclass
class Message_state:
    messages: list[ModelMessage]
@dataclass
class Deps:
    query: str
    #planning notes are notes to improve the planning or use of a tool based on a prompt
    planning_notes:str
    #query notes are notes to help the agent to fulfill the requirements of the user
    query_notes:dict

class Notion_agent:
    def __init__(self, api_keys:dict):
        """
        Args:
            
            api_keys (dict): The API keys to use
            
        """
        self.llms={'pydantic_llm':GoogleModel('gemini-2.5-flash-preview-05-20', provider=GoogleProvider(api_key=api_keys['google_api_key'])),
              
              'openai_llm':ChatOpenAI(model='gpt-4.1-nano',api_key=api_keys['openai_api_key'])}   
        # tools is the composio toolset
        self.tools=ComposioToolSet(api_key=api_keys['composio_key'])
        # tool_shemas is a dictionary of the tool names and the actions they can perform
        self.tool_shemas={
            'Notion Manager':{tool.name:tool for tool in self.tools.get_action_schemas(apps=[App.NOTION])},
            'Get_current_time':{'get_current_time':'get the current time'},
            'Planning_notes_editor':{'planning_notes_editor':'notes to improve the planning or use of a tool based on a prompt'},
            'Query_notes_editor':{'query_notes_editor':'edit the query notes to fulfill the requirements of the tool'}
        }
        # tool_functions is a dictionary of the tool names and the actions they can perform
        self.tool_functions={
            'managers':{
                'Notion Manager':{
                    'actions':{tool.name:{'description':tool.description} for tool in self.tools.get_tools(apps=[App.NOTION])}
                },
                'Planning_notes_editor':{
                    'actions':{'planning_notes_editor':{'description':'notes to improve the planning or use of a tool based on a prompt'}}
                },
                'Get_current_time':{
                    'actions':{'get_current_time':{'description':'get the current time'}}
                },

                'Query_notes_editor':{
                    'actions':{'query_notes_editor':{'description':'edit the query notes to fulfill the requirements of the manager tool'}}
                },
            }
        }
        # agents are the composio agents for the tools
        self.notion_agent=Composio_agent(self.tools.get_tools(apps=[App.NOTION]),self.llms['openai_llm'])


        async def planning_improve_tool(ctx: RunContext[Deps],query:str):
            class planning_improve_shema(BaseModel):
                planning_improvement: str = Field(description='the planning improvement notes')
            agent=Agent(self.llms['pydantic_llm'],output_type=planning_improve_shema, instructions=f'based on the dict of tools and the prompt, and the previous planning notes (if any), create a notes to improve the planning or use of a tool for the planner node')
            response=agent.run_sync(f'prompt:{query}, tool_functions:{format_as_xml(self.tool_functions)}, previous_planning_notes:{ctx.deps.planning_notes if ctx.deps.planning_notes else "no previous planning notes"}')
            ctx.deps.planning_notes=response.output.planning_improvement
            
            return response.output.planning_improvement
        
 
        async def query_notes_tool(ctx: RunContext[Deps],query:str):
            class query_notes_shema(BaseModel):
                query_notes: str = Field(description='the query notes has to be an explanation of how to use the tool to complete the task')
                manager_tool: str = Field(description='the name of the manager tool for the query')
                action: str = Field(description='the action that the manager tool must take')

            agent=Agent(self.llms['pydantic_llm'],output_type=query_notes_shema, instructions=f'based on the user query, and the tools, edit the query notes to help the agent to fulfill the requirements of the user')
            response=agent.run_sync(f'prompt:{query}, tool_functions:{format_as_xml(self.tool_functions)}')
            if ctx.deps.query_notes.get(response.output.manager_tool):
                ctx.deps.query_notes[response.output.manager_tool][response.output.action]={'query_notes':response.output.query_notes}
            else:
                ctx.deps.query_notes[response.output.manager_tool]={response.output.action:{'query_notes':response.output.query_notes}}

            return response.output.query_notes

       
        
        async def get_current_time_tool(ctx: RunContext[Deps]):
            """
            Use this tool to get the current time.
            Returns:
                str: The current time in a formatted string
            """
            return f"The current time is {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        
        async def notion_manager_tool(ctx: RunContext[Deps],query:str):
            f""" 
            Use this tool to manage the Notion database.
            the actions are:
            {self.tool_shemas['Notion Manager']}
            Args:
                query (str): The query to manage the Notion database.
            Returns:
                dict: The response from the Notion agent.
            """
            response=self.notion_agent.chat(query)
            return response




        self.memory=Message_state(messages=[])
        self.deps=Deps(query='', planning_notes='', query_notes={})
        self.agent=Agent(self.llms['pydantic_llm'],tools=[get_current_time_tool, planning_improve_tool, query_notes_tool, notion_manager_tool], system_prompt=f'You are a helpful Notion database assistant that can use tools to help the user ')
        
    def chat(self,query:str):
        """Chat with the notion agent,
        Args:
            query (str): The query to search for
        Returns:
            dict: The state of the notion agent
        """
        self.deps.query=query
        response=self.agent.run_sync(query, deps=self.deps, message_history=self.memory.messages)
        self.memory.messages=response.all_messages()
        
        return response.output


    def reset(self):
        """Reset the state of the Notion agent
        """
        self.deps=Deps(query='', planning_notes='', query_notes={})
        self.memory.messages=[]
        return 'agent reset'

In [26]:
agent=Notion_agent(api_keys)

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Notion-agent\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions


  warnings.warn(


In [27]:
agent.tool_shemas

{'Notion Manager': {'NOTION_ADD_PAGE_CONTENT': ActionModel(name='NOTION_ADD_PAGE_CONTENT', description='Appends a single content block to a notion page or a parent block (must be page, toggle, to-do, bulleted/numbered list, callout, or quote); invoke repeatedly to add multiple blocks.', parameters=ActionParametersModel(properties={'after': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': "Identifier of an existing block. The new content block will be appended immediately after this block. If omitted or null, the new block is appended to the end of the parent's children list. Please provide a value of type string.", 'examples': ['4b5f6e87-123a-456b-789c-9de8f7a9e4c0'], 'title': 'After', 'type': 'string'}, 'content_block': {'description': "A NotionRichText object defining the content and type of the new block to be added (e.g., paragraph, heading, to_do). This structure must conform to Notion's block object schema for child content. Refer to Notion API do

In [29]:
res=agent.chat("use the fetch data to get it")
print(res)

To fetch the data from your notes database, I still need the `database_id`. Could you please provide the `database_id` for your notes database?


In [15]:
from pydantic_ai.mcp import MCPServerStreamableHTTP
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai import Agent
from pydantic_ai.messages import ModelMessage
from dataclasses import dataclass

@dataclass
class Message_state:
    messages:list[ModelMessage]

class NotionAgent_MCP:
    def __init__(self, mcp_server_url:str, api_keys:dict):
        self.mcp_server_url=mcp_server_url
        self.api_keys=api_keys
        self.tools=ComposioToolSet(api_key=api_keys['composio_key'])
        self.tool_shemas={
            'Notion Manager':{tool.name:tool for tool in self.tools.get_action_schemas(apps=[App.NOTION])}}
        self.memory=Message_state(messages=[])
        self.llm=OpenAIModel('gpt-4.1-nano',provider=OpenAIProvider(api_key=api_keys['openai_api_key']))
        self.mcp_server=MCPServerStreamableHTTP(self.mcp_server_url)
        self.agent=Agent(self.llm, mcp_servers=[self.mcp_server])

    async def main(self,query:str):
        async with self.agent.run_mcp_servers():  
            result = self.agent.run_sync(query, message_history=self.memory.messages)
            self.memory.messages=result.all_messages()
        return result.output

    def reset(self):
        self.memory.messages=[]

In [16]:
agent2=NotionAgent_MCP(mcp_server_url='https://mcp.composio.dev/composio/server/fa68ab92-4f30-48a3-8430-26b971efa5a1/mcp', api_keys=api_keys)

In [24]:
await agent2.main('update it to add contents: "this is a test"')

'The content of the note titled "test" has been successfully updated to include the text: "this is a test". If you need further modifications or assistance, just let me know.'

In [1]:
from notion_agent import Notionagent

ImportError: cannot import name 'Notionagent' from 'notionmanager' (consider renaming 'c:\\Users\\trist\\OneDrive\\Desktop\\important\\ai_portfolio\\Notion-agent\\notionmanager.py' if it has the same name as a library you intended to import)